- tại sao cột reviewdate của pl04 null - xem lại tiền xử lí cho cột reviewdate của youtube
- 303 dòng null trong cột review (gồm cả post và comment - pl004 youtube -> samsung)
- 236 dòng của cột tilte và reviewername,commentcount null (typereview_comment - pl005- tinhte - product 02- samsung)
- title youtube: VinhXo , if TITLE.STARTWITH (Vinh Xô |) -> remove startwith..
- drop duplicated

In [1]:
import pandas as pd
from datetime import datetime
import dateutil.relativedelta
from datetime import date
# import pypyodbc as odbc
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)

# Test database

In [2]:
# drive_name = 'SQL SERVER'
# server_name = 'DESKTOP-29355SM\SQLEXPRESS'
# database_name = 'Bank'

# connection_string = f"""
#     DRIVER={{{drive_name}}};
#     SERVER={server_name};
#     DATABASE={database_name};
#     Trust_Connection=yes;
# """

# conn = odbc.connect(connection_string)

In [3]:
# cursor = conn.cursor()

In [4]:
# cursor.execute("""
# Create table Persons (
#     PersonID int,
#     LastName varchar(255),
#     FirstName varchar(255),
#     Address varchar(255),
#     City varchar(255)
# )
# """)

In [5]:
# df = pd.read_sql('Select * from account', conn)

In [6]:
# conn.commit()

# Function

In [2]:
def extract_date_review(text_date, crawl_date,key=None): # Biến đổi khoảng thời gian (1 tháng trước, 2 ngày trước) thằnh ngày cụ thể và trả về datetime
    sub_text = text_date.split(' ')
    date_cmt = ''
    if 'năm' in text_date:
        if key is None:
            for t in sub_text:
                if t.isnumeric():
                    date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(t))
        elif key =='tinhte':
            for t in sub_text:
                if t=="một":
                    date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(1))
                elif t=="hai":
                    date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(2))
    elif 'tháng' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(t))
            elif t =="một":
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(1))
    elif 'tuần' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(weeks==int(t))
    elif 'ngày' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(days=int(t))
    else:
        date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date()

    return date_cmt

In [3]:
def create_id(last_id): # Tạo ID mới và trả về string
    last_id_list = []
    newest_id = ''

    for s in list(last_id):
        if s.isnumeric():
            last_id_list.append(s)
        else:
            newest_id += s
    
    last_id = str(int(''.join(last_id_list)) + 1)
    if len(last_id_list) != len(last_id):
        num_0 = len(last_id_list) - len(last_id)
        newest_id += '0'*num_0 + last_id
    else:
        newest_id += last_id
    
    return newest_id

In [4]:
def add_data(file_table_name, df, id_format=None): # Thêm df vào file save
    # Lưu ý: df không chứa cột ID, thứ tự cột và định dạng giống với file save và nếu file save trống thì cần đưa ID format (S000, PL00,...)
    table = pd.read_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name))

    # Loại bỏ những dòng trong df đã có trong table
    drop_index = []
    for index in range(len(df)):
        if list(df.iloc[index]) in table[table.columns[1:]].values.tolist():
            drop_index.append(index)
    df.drop(drop_index, inplace=True)

    newest_id = ''
    if id_format != None:
        newest_id = create_id(id_format)
    else:
        newest_id = create_id(table.iloc[-1][0])

    list_id = []
    for i in range(len(df)):
        list_id.append(newest_id)
        newest_id = create_id(newest_id)
    df.insert(loc=0, column='ID', value=list_id)

    df.columns = table.columns
    table = pd.concat([table, df], ignore_index=True)
    table.to_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name), index=False)

In [5]:
def add_shop(new_shop_df): # Thêm shop mới vào file Shop.csv (new_shop_df chỉ bao gồm 'Shop name', 'Shop rating' và đặt đúng thứ tự)
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    check_shop_list = new_shop_df.drop_duplicates(subset=new_shop_df.columns[0], keep='last', ignore_index=True)
    drop_index = []

    for index_shop in range(len(check_shop_list)):
        if check_shop_list.iloc[index_shop][0] in shop_df['ShopName'].values:
            drop_index.append(index_shop)

    check_shop_list.drop(drop_index, inplace=True)

    if len(shop_df) == 0:
        add_data('Shop.csv', check_shop_list, id_format='S000')
    else:
        add_data('Shop.csv', check_shop_list)

# Main

In [11]:
# product = input()
product = 'iphone 15 pro max'

# Tiki

#### Product info

In [12]:
tiki_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product.split(' '))))

In [13]:
# Lấy những cột cần thiết
tiki_product_info = tiki_product_info[['Name', 'Brand', 'Quantity', 'Reviews count', 'Star rating', 
                                       'Price', 'Crawl date', 'Shop name', 'Shop rating']]

In [14]:
tiki_product_info.dtypes

Name              object
Brand             object
Quantity           int64
Reviews count      int64
Star rating      float64
Price             object
Crawl date        object
Shop name         object
Shop rating      float64
dtype: object

In [15]:
# Thêm cột "Platform ID"
tiki_product_info['Platform ID'] = 'PL01'

In [ ]:
# Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
tiki_product_info['Price'] = tiki_product_info['Price'].str.replace('.', '', regex=True).astype(int)

In [ ]:
tiki_product_info

,Name,Brand,Quantity,Reviews count,Star rating,Price,Crawl date,Shop name,Shop rating,Platform ID
0,Apple iPhone 15 Pro Max,Apple,691,96,5.0,31990000,2024-01-11,Tiki Trading,4.7,PL01
1,Apple iPhone 15 Pro Max,Apple,726,100,5.0,31990000,2024-01-18,Tiki Trading,4.7,PL01
2,Apple iPhone 15 Pro Max,Apple,740,103,5.0,31990000,2024-01-27,Tiki Trading,4.7,PL01


In [ ]:
tiki_product_info.dtypes

Name              object
Brand             object
Quantity           int64
Reviews count      int64
Star rating      float64
Price              int32
Crawl date        object
Shop name         object
Shop rating      float64
Platform ID       object
dtype: object

#### Reviews

In [ ]:
tiki_reviews = pd.read_csv(r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product.split(' '))))

In [ ]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Crawl date,Shop name
0,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
1,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
2,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
3,NGÔ QUỐC KHÁNH,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
4,Hung Trieu,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,Cực kì hài lòng,Đánh giá vào 2 tháng trước,2024-01-11,Tiki Trading
...,...,...,...,...,...,...
294,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,Rất không hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading
295,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,Rất không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
296,Thu Nga,Chưa thấy hoàn astra cho mình,Không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
297,Nguyễn Quang,NaN,Hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading


In [ ]:
# Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
actual_date_list = []

for num in range(len(tiki_reviews)):
    text_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][0]
    crawl_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][1]

    actual_date = extract_date_review(text_date, crawl_date)
    actual_date_list.append(actual_date)

tiki_reviews['Rating date'] = actual_date_list
tiki_reviews.drop(columns=['Crawl date'], inplace=True)

In [ ]:
# Biến đổi cột "Rating" thành thang đo likert
rating_likert = {'Rất không hài lòng': 1, 'Không hài lòng': 2, 'Bình thường': 3, 'Hài lòng': 4, 'Cực kì hài lòng': 5}

for num in range(len(tiki_reviews)):
    rating = tiki_reviews.loc[num, 'Rating']
    for k, v in rating_likert.items():
        if rating == k:
            tiki_reviews.loc[num, 'Rating'] = v

In [ ]:
# Loại bỏ những dòng trùng "Reviewer name" và "Content"
tiki_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

In [ ]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Shop name
0,Lý Nguyen,😂. Haha,5,2023-12-23,Tiki Trading
1,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-12-11,Tiki Trading
2,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",5,2023-09-27,Tiki Trading
3,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,5,2023-09-27,Tiki Trading
4,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,5,2023-09-27,Tiki Trading
...,...,...,...,...,...
96,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-11-27,Tiki Trading
97,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,1,2023-10-27,Tiki Trading
98,Thu Nga,Chưa thấy hoàn astra cho mình,2,2023-10-27,Tiki Trading
99,Nguyễn Quang,NaN,4,2023-11-27,Tiki Trading


# Lazada

# Websites

In [6]:

# Function
def fill_nan_values(df,key): 
    # Fill NaN when combined comment and post data
    url_groups = df.groupby(key)
    for _,group in url_groups:
        non_nan_row = group.dropna(subset=['Title','ReviewerName','CommentCount'])
        if non_nan_row.empty:
            continue
        non_nan_row = non_nan_row.iloc[0]
        nan_rows_index = group.index[group[['Title','ReviewerName','CommentCount']].isnull().any(axis=1)].tolist()
        for index in nan_rows_index:
            df.loc[index,'Title']  =non_nan_row['Title']
            df.loc[index,'ReviewerName'] = non_nan_row['ReviewerName']
            df.loc[index,'CommentCount'] = non_nan_row['CommentCount']
            if key=='Id':
                df.loc[index,'ViewCount'] = non_nan_row['ViewCount']


    return df


#### Tinhte _Uncompleted
 -- Handing NaN values: finding causes

In [7]:
# create a function to remove non content pages (page2, page3...) -- for post data OR edit link (xx/page2;xx/page3 -> xx/) -- for comment data
def remove_or_edit_links(data,type):
    if type.lower() == 'post':
        del_rows=0
        for index,link in data['Link'].items():
            last_link = link.split('/')[-1]
            if last_link !='':
                data.drop(index,inplace=True)
                del_rows+=1
        print(f'Deleted: {del_rows} rows')

    elif type.lower() =='cmt' or type.lower()=='comment':
        edited_rows=0
        for index,link in data['Link'].items():
            last_link =  link.split('/')
            if last_link[-1] !='':
                del(last_link[-1])
                newlink = '/'.join(last_link) +'/'
                data['Link'].iloc[index] = newlink
                edited_rows+=1

        print(f'Edited: {edited_rows} links')

    return data

In [30]:


# provide keyword items folowing index[Nameproduct,Product_id,crawldate]
tt_keyword = [['Iphone15pm','P001'],['SamsungS23u','P002']]

df = pd.DataFrame()

for i in range(len(tt_keyword)):
    
    name_info= pd.read_csv(f'..\Data\TinhTe\TinhTe_{tt_keyword[i][0]}_InfoPosts.csv')
    name_cmt = pd.read_csv(f'..\Data\TinhTe\TinhTe_{tt_keyword[i][0]}_Comments.csv')

    # Convert Date columns to the same format: %Y-%m-%d
    name_info['Post Date'] = pd.to_datetime(name_info['Post Date'],format='%d/%m/%Y %H:%M')
    name_info['Post Date'] = name_info['Post Date'].dt.strftime('%Y-%m-%d')
    name_cmt['Crawl date'] = date(2024,1,19).strftime('%Y-%m-%d')

    actual_date_list = []

    for num in range(len(name_cmt)):
        text_date = name_cmt.loc[num, ['Date', 'Crawl date']][0]
        crawl_date = name_cmt.loc[num, ['Date', 'Crawl date']][1]
        actual_date = extract_date_review(text_date, crawl_date,'tinhte')
        actual_date_list.append(actual_date)

    name_cmt['Date'] = actual_date_list
    name_cmt.drop(columns=['Crawl date'],inplace=True)

    # Link Column 
    name_info = remove_or_edit_links(name_info,'post')
    name_cmt = remove_or_edit_links(name_cmt,'cmt')

    # Rename columns following the required format in the Database
    name_info.rename(columns={'Author':'ReviewerName','Post Date':'DateReview','Comments Count':'CommentCount','Content':'Review'},inplace=True)
    name_cmt.rename(columns={'Date':'DateReview','Comment':'Review'},inplace=True)

    # Add PlatformID column
    name_info['PlatformID'] = 'PL05'
    name_cmt['PlatformID'] = 'PL05'

    # Add ProductID column
    name_info['ProductID'] = tt_keyword[i][1]
    name_cmt['ProductID'] = tt_keyword[i][1]

    # Add TypeReview column
    name_info['TypeReview'] = 'Post'
    name_cmt['TypeReview'] = 'Comment'
    
    # Preprocess date column of samsung comment  data again
    name_info['DateReview'] = pd.to_datetime(name_info['DateReview']) 
    name_cmt['DateReview'] = pd.to_datetime(name_cmt['DateReview']) 

    # Add data to df - summary file
    df = pd.concat([df,name_info,name_cmt],ignore_index=True)

df = fill_nan_values(df,'Link')

#Drop unneccesary columns
df.drop(columns=['Link'], inplace=True)
df['ViewCount'] = None
desired_columns=['Review', 'ProductID', 'PlatformID', 'TypeReview',
                'DateReview', 'Title', 'ReviewerName', 'ViewCount', 'CommentCount']
df=df[desired_columns]

# Check data
df['DateReview'] = pd.to_datetime(df['DateReview']) 
df['ViewCount'] = df['ViewCount'].astype(float)  
df['CommentCount'] = df['CommentCount'].astype(float)
print(df.isna().sum())

# Save data
smr_df = pd.read_csv(r'..\Data\Preprocessed_data\SocialMediaReviews.csv')
print('Number rows inserted:',len(df))
if len(smr_df) == 0:
    add_data('SocialMediaReviews.csv', df, 'SM00000000')
else:
    add_data('SocialMediaReviews.csv', df)

Deleted: 0 rows
Edited: 0 links
Deleted: 24 rows
Edited: 626 links
Review             26
ProductID           0
PlatformID          0
TypeReview          0
DateReview          0
Title             235
ReviewerName      235
ViewCount       15326
CommentCount      235
dtype: int64
Number rows inserted: 15326


# Youtube -- reply comments preprocessing
- reply comments
- convert comment(cmt) , date published(info) to the general format
- group comment id to fill null values in comment data (cmt)


In [8]:
yt_keyword = [['iphone_15_pro_max','P001'],['galaxy_s23_ultra','P002']]

# A function to preprocess and remove usertag in reply comment column
def reply_comments(yt_cmt,name_data):
    count=0
    print(f'Initial {name_data} comment data include: {len(yt_cmt)} rows')
    for index,reply in  yt_cmt['Reply'].items():
        if reply is not None and not isinstance(reply, float) and not isinstance(reply, int):
            reply = reply.split('<>')
            for rl in reply:
                # Remove tag user: @@name_user
                if rl.startswith("@@"):
                    lst_rl = rl.split()
                    rl = ' '.join(lst_rl[1:])
                count+=1
                new_row_df = {'Unnamed: 0':None,'Id':yt_cmt.loc[index,'Id'],'Comment':rl,'Comment time':yt_cmt.loc[index,'Comment time'],'Reply':None}
                yt_cmt.loc[len(yt_cmt)] = new_row_df
    print('Number rows inserted:',count)
    print(f'After preprocessing [Reply] column, {name_data} comment data include: {len(yt_cmt)} rows')
    return yt_cmt

def yt_preprocessing():
    yt_df = pd.DataFrame()
    for i in range(len(yt_keyword)):

        #Import data
        yt_info = pd.read_csv(f'../Data/Youtube/video_info_{yt_keyword[i][0]}.csv')
        yt_cmt = pd.read_csv(f'../Data/Youtube/comments_{yt_keyword[i][0]}.csv')
        
        # Preprocessing reply cmts
        yt_cmt = reply_comments(yt_cmt,yt_keyword[i][0])
        
        # Rename columns
        yt_info.rename(columns={'Date published':'DateReview','Channel title':'ReviewerName',
                                'Comments count':'CommentCount','View count':'ViewCount'},inplace=True)
        yt_cmt.rename(columns={'Comment':'Review','Comment time':'DateReview'},inplace=True)

        # Add ProductID column
        yt_info['ProductID'] = yt_keyword[i][1]
        yt_cmt['ProductID'] = yt_keyword[i][1]

        # Add TypeReview column
        yt_info['TypeReview'] = 'Post'
        yt_cmt['TypeReview'] = 'Comment'
        # Add data to main df
        yt_df = pd.concat([yt_df,yt_info,yt_cmt],ignore_index=True)

    return yt_df


def yt_main(df):
    df = fill_nan_values(df,'Id')
    # Remove unneccesary columns
    df.drop(columns=['Unnamed: 0','Like count','Id'],inplace=True)
    # Add PlatformID column 
    df['PlatformID'] ='PL04'
    # Convert DateReview column to the general format
    df['DateReview'] = pd.to_datetime(df['DateReview'],format="%Y-%m-%dT%H:%M:%SZ")
    df['DateReview'] = df['DateReview'].dt.strftime('%Y-%m-%d')
    df['ViewCount'] = df['ViewCount'].astype(float) 
    df['CommentCount'] = df['CommentCount'].astype(float)
    print('-----------------------------------------------------')
    print('Check number of null rows /columns in dataframe: ')
    print(df.isna().sum())
    
    return df



In [27]:
pre_df= yt_preprocessing()
yt_df = yt_main(pre_df)

# Save file
desired_columns=['Review', 'ProductID', 'PlatformID', 'TypeReview',
                'DateReview', 'Title', 'ReviewerName', 'ViewCount', 'CommentCount']
yt_df=yt_df[desired_columns]
print(f'After all preprocessing basic steps, Youtube data include {len(yt_df)} rows.')
print(yt_df.columns)
smr_df = pd.read_csv(r'..\Data\Preprocessed_data\SocialMediaReviews.csv')
if len(smr_df) == 0:
    add_data('SocialMediaReviews.csv', yt_df, 'SM00000000')
else:
    add_data('SocialMediaReviews.csv', yt_df)


Initial iphone_15_pro_max comment data include: 15233 rows
Number rows inserted: 9289
After preprocessing [Reply] column, iphone_15_pro_max comment data include: 24522 rows
Initial galaxy_s23_ultra comment data include: 11429 rows
Number rows inserted: 9264
After preprocessing [Reply] column, galaxy_s23_ultra comment data include: 20693 rows
-----------------------------------------------------
Check number of null rows /columns in dataframe: 
DateReview          0
Title               0
ReviewerName        0
ViewCount           0
CommentCount        0
ProductID           0
TypeReview          0
Review            255
Reply           37542
PlatformID          0
dtype: int64
After all preprocessing basic steps, Youtube data include 45466 rows.
Index(['Review', 'ProductID', 'PlatformID', 'TypeReview', 'DateReview',
       'Title', 'ReviewerName', 'ViewCount', 'CommentCount'],
      dtype='object')


C:\Users\nguye\AppData\Local\Temp\ipykernel_18072\3735578254.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(drop_index, inplace=True)


#### Check len() of socialmediareviews

In [33]:
df_smm = pd.read_csv('../Data/Preprocessed_data/SocialMediaReviews.csv')
print(df_smm.isna().sum())
print(len(df_smm))

SMReviewID          0
Review            302
ProductID           0
PlatformID          0
TypeReview          0
DateReview          0
Title             235
ReviewerName      235
ViewCount       15326
CommentCount      235
dtype: int64
60792


# Reset data saved
df_smm = pd.read_csv('../Data/Preprocessed_data/SocialMediaReviews.csv')
print(df_smm.isna().sum())
columns = df_smm.columns

empty_df = pd.DataFrame(columns=columns)

# Save the empty DataFrame back to the CSV file, overwriting it
empty_df.to_csv('../Data/Preprocessed_data/SocialMediaReviews.csv', index=False)

print("Data removed from the CSV file, only column headers remain.")